In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [2]:
df = pd.read_csv('./data/Telco Customer Churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
# print(df.head())
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# errors='coerce' is a good approach to handle non-numeric values by replacing them with NaN.


In [11]:
def fill_missing_values(df, option : str='mean'):
    """Fill missing values in data frame"""
    
    missing_values = df.isna().sum()
    # print(missing_values)

    # Check if there are any missing values
    if missing_values.any():
        print("There are missing values in the dataset.")
        # Display the count of missing values for each column
        # find the columns with missing values
        columns_with_missing_values = df.columns[missing_values > 0]
        print("Missing values per column:")
        print(missing_values[columns_with_missing_values])

        # fix the missing values
        # we can use df.replace(to_replace=' ', value=-1) to replace all missing values with -1
        for column in columns_with_missing_values:
       
            if option == 'drop':
                df_new = df.dropna()
            elif option == 'mean':
                df_new = df[column].fillna(df[column].mean())
            elif option == 'median':
                df_new = df[column].fillna(df[column].median())
            elif option == 'mode':
                df_new = df[column].fillna(df[column].mode()[0])
            elif option == 'linear':
                df_new = df[column].interpolate(method='linear', limit_direction='forward')
            elif option == 'quadratic':
                df_new = df[column].interpolate(method='quadratic', limit_direction='forward')
            elif option == 'cubic':
                df_new = df[column].interpolate(method='cubic', limit_direction='forward')
            elif option == 'spline':
                df_new = df[column].interpolate(method='spline', order=3, limit_direction='forward')
        
        print("Missing values per column are fixed\n")

        return df_new

    else:
        print("There are no missing values in the dataset.\n")
        return df


In [12]:
learn_df = df.copy()

learn_df = fill_missing_values(df, option='mean')



numeric_cols = learn_df.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove("SeniorCitizen")
categorical_cols = learn_df.select_dtypes(exclude=np.number).columns.tolist()

print(numeric_cols)

# Normalization for numeric columns
scaler = StandardScaler()
learn_df[numeric_cols] = scaler.fit_transform(learn_df[numeric_cols])

# Label Encoding for categorical columns
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    learn_df[col] = label_encoders[col].fit_transform(learn_df[col])

There are missing values in the dataset.
Missing values per column:
TotalCharges    11
dtype: int64
Missing values per column are fixed



AttributeError: 'Series' object has no attribute 'select_dtypes'

In [119]:
learn_df.head()

0      29.85
1    1889.50
2     108.15
3    1840.75
4     151.65
Name: TotalCharges, dtype: float64

In [110]:
# Ranking features
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

# split the last column of the array fot the label array and the rest for training
X = learn_df.drop(columns=['Churn']) # features
y = learn_df['Churn'] # label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=82)
# random_state=82 is used to ensure that the split is always the same

# rank features using mutual_info_classif
selector = SelectKBest(mutual_info_classif, k=5)
selector.fit(X_train, y_train)
scores = selector.scores_
scores = pd.DataFrame(scores, index=X_train.columns, columns=['Score'])
scores.sort_values(by='Score', ascending=False, inplace=True)
scores



ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values